This colab is the chat only version of this notebook: [bit.ly/gemma-pirate-demo](https://bit.ly/gemma-pirate-demo)
##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/get_started"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Acesse ai.google.dev</a>
  </td>
    <td>
    <a target="_blank" href="https://colab.research.google.com/github.com/lucianommartins/generative-ai-ptbr/blob/main/gemma/2_chatbot_with_Gemma.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Execute este notebook no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/lucianommartins/generative-ai-ptbr/blob/main/gemma/2_chatbot_with_Gemma.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Execute este notebook na Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/lucianommartins/generative-ai-ptbr/blob/main/gemma/2_chatbot_with_Gemma.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Consulte o código no GitHub</a>
  </td>
</table>

# Criando um chatbot com modelos Gemma

Este tutorial mostra como criar um chatbot utilizando os modelos Google Gemma. LLMs são muito bons para criar assistentes ou chatbots porque podem dar respostas muito boas e uma conversa com um LLM só faz sentido quando ele consegue lembrar o que foi perguntado antes.

**Importante:** Para uma melhor experiência de execução deste laboratório, é recomendado que se utilize uma máquina com aceleradores (GPU ou TPU). Em uma máquina sem aceleradores (como GPU) esta geração de conteúdo pode levar cerca de 80 segundos para ser concluída.

## Configurando o ambiente

### Instalando dependências

Instale os pacotes `pip` Keras e KerasNLP.

In [ ]:
# Instale a última versão do Keras 3. Acesse https://keras.io/getting_started/ para maiores detalhes.
!pip install -U keras-nlp
!pip install -U keras>=3
!pip install -U jax
!pip install -U jaxlib

### Configurações para o uso do Gemma

Para concluir este tutorial, primeiro você precisa seguir as instruções de configuração em [Configuração do Gemma](https://ai.google.dev/gemma/docs/setup). As instruções de configuração do Gemma mostram como fazer o seguinte:

* Obtenha acesso a Gemma em kaggle.com.
* Selecione um tempo de execução do Colab com recursos suficientes para execução
   o modelo Gemma 2B.
* Gere e configure um nome de usuário Kaggle e uma chave de API.

Depois de concluir a configuração do Gemma, vá para a próxima seção, onde você definirá variáveis de ambiente para seu ambiente Colab.

**TL;DR: Acesse https://www.kaggle.com/models/google/gemma e faça a solicitação de acesso ao modelo Gemma**

### Configurando variáveis de ambiente necessárias para utilizar o Kaggle

Configure as variáveis `KAGGLE_USERNAME` (contendo seu usuário Kaggle) e `KAGGLE_KEY` (com sua chave de acesso ao Kaggle).

In [ ]:
import os

# Nota: `userdata.get` é uma API Colab. Se você não estiver usando o Colab, defina o ambiente
# vars conforme apropriado para seu sistema.

# descomente o par de linhas que faça mais sentido pra você:
# # opção A: utilizando direto no Colab
# from google.colab import userdata
# os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
# os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

# # opção B: utilizando na Vertex AI ou no seu ambiente pessoal
# os.environ["KAGGLE_USERNAME"] = "seu usuário"
# os.environ["KAGGLE_KEY"] = "sua chave de autenticação"

### Escolhe o backend a ser utilizado com o Keras

Keras é uma API de deep learning multi-backend de alto nível projetada para simplicidade e facilidade de uso. [Keras 3](https://keras.io/keras_3) permite escolher o backend: TensorFlow, JAX ou PyTorch. Todos os três funcionarão para este tutorial.

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"

# pré-aloque 100% da memóeria da TPU pra diminuir fragmentação de memória
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

import keras
import keras_nlp

# por reproducibilidade
keras.utils.set_random_seed(42)

## Defina um modelo

KerasNLP fornece implementações de muitas [arquiteturas de modelos populares](https://keras.io/api/keras_nlp/models/). Neste tutorial, você criará um modelo usando `GemmaCausalLM`, um modelo Gemma ponta a ponta para modelagem de linguagem causal. Um modelo de linguagem causal prevê o próximo token com base nos tokens anteriores.

Você usará a versão Afinada com Instrução porque é a versão que foi preparada para manter conversas e tirar dúvidas de forma mais natural.

Crie o modelo usando o método `from_preset`:

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

### Uma função de apoio para facilitar a formatação das interações

In [ ]:
# formatting utility
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Vamos conversar

Carregamos o modelo ajustado por instrução `gemma_instruct_2b_en`, que compreende os seguintes tokens de turno:
```
<start_of_turn>usuário\n  ... <end_of_turn>\n
<start_of_turn>modelo\n ... <end_of_turn>\n
```

### Função de apoio para o chat (para gerenciar o estado da discussão)

In [ ]:
class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines of the Gemma family of models.
  documented here: https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message + self.__END_TURN__)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=1024)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result


In [ ]:
chat = ChatState(gemma_lm)
message = "Diga-me, em poucas palavras, como calcular todos os números primos até 1000?"
display_chat(message, chat.send_message(message))

In [ ]:
message = "Agora em Python! Sem usar Numpy, por favor!"
display_chat(message, chat.send_message(message))

In [ ]:
message = "Obrigado, assim funciona. Pode me explicar esse código?"
display_chat(message, chat.send_message(message))

In [ ]:
message = "Massa! Agora adicione essas explicações como comentários no código."
display_chat(message, chat.send_message(message))

Vamos tentar a resposta gerada. Apenas copiar o código para a próxima célula e executá-lo deve nos dar as respostas corretas:

In [ ]:
def is_prime(n):
  """
  Checks if a number is prime.

  Args:
    n: The number to check.

  Returns:
    True if n is prime, False otherwise.
  """

  # If n is less than or equal to 1, it is not prime.
  if n <= 1:
    return False

  # Iterate through all the numbers from 2 to the square root of n.
  for i in range(2, int(n**0.5) + 1):
    # If n is divisible by any of the numbers in the range from 2 to the square root of n, it is not prime.
    if n % i == 0:
      return False

  # If no divisors are found, n is prime.
  return True


# Initialize an empty list to store prime numbers.
primes = []

# Iterate through all the numbers from 2 to 1000.
for i in range(2, 1001):
  # If the number is prime, add it to the list.
  if is_prime(i):
    primes.append(i)

# Print the prime numbers.
print(primes)

você também pode ver como todo o contexto foi mantido pela classe Chat, mas chamando o método `get_history`:

In [ ]:
print(chat.get_history())

# Próximos passos

Neste tutorial, você aprendeu como conversar com o modelo ajustado de instrução Gemma 2B usando Keras no JAX.

Na próxima etapa, você pode ajustar o modelo para ter um tom ou voz específico e usar esse modelo na classe de bate-papo.

Aqui estão algumas sugestões sobre como ajustar o modelo usando Keras e JAX:
* [Treinamento distribuído com Keras 3](https://keras.io/guides/distribution/).
* [Escrever um loop de treinamento personalizado para um modelo Keras em JAX](https://keras.io/guides/writing_a_custom_training_loop_in_jax/).

E mais alguns tutoriais básicos do Gemma:

* [Primeiros passos com Keras Gemma](https://ai.google.dev/gemma/docs/get_started).
* [Tuning do modelo Gemma na GPU](https://ai.google.dev/gemma/docs/lora_tuning).
* Saiba mais sobre [integração do Gemma com Vertex AI](https://ai.google.dev/gemma/docs/integrations/vertex)
* Saiba como [usar modelos Gemma com Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).